In [ ]:
import requests
from PIL import Image
from transformers import BlipProcessor, BlipForQuestionAnswering

processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-capfilt-large")
model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-capfilt-large")




In [ ]:
import cv2
import requests
from PIL import Image
from transformers import BlipProcessor, BlipForQuestionAnswering

In [ ]:
img_url = 'https://storage.googleapis.com/sfr-vision-language-research/BLIP/demo.jpg' 
raw_image = Image.open(requests.get(img_url, stream=True).raw).convert('RGB')

question = "what color is cat in the image?"
inputs = processor(raw_image, question, return_tensors="pt")

out = model.generate(**inputs)
print(processor.decode(out[0], skip_special_tokens=True))

In [3]:
from moviepy.editor import VideoFileClip

def convert_video_to_audio(video_path, audio_path):
    video_clip = VideoFileClip(video_path)
    audio_clip = video_clip.audio
    audio_clip.write_audiofile(audio_path)
    audio_clip.close()
    video_clip.close()
 

convert_video_to_audio("w.MOV", "state.mp3")


MoviePy - Writing audio in state.mp3


MoviePy - Done.


In [ ]:
!python3 -m pip install ffmpeg

In [5]:
from openai import OpenAI

client = OpenAI(api_key="")


In [ ]:
import cv2
import base64

def encode_image(frame):
    # Convert the frame to JPEG format
    _, buffer = cv2.imencode('.jpg', frame)
    # Encode the frame as base64 string
    return base64.b64encode(buffer).decode('utf-8')

def summarize_text():
    video_path = 'video.mp4'
    cap = cv2.VideoCapture(video_path)

    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    images_array = []

    for second in range(0, int(frame_count // fps), 5):  # Iterate every 5 seconds
        cap.set(cv2.CAP_PROP_POS_FRAMES, int(second * fps))
        
        success, frame = cap.read()
        if not success:
            break
        
        # Encode the frame as base64 and append to the array
        base64_image = encode_image(frame)
        images_array.append({
            "type": "image_url",
            "image_url": {
                "url": f"data:image/jpeg;base64,{base64_image}"
            }
        })

    cap.release()
    max_num = 11
    frame_text = ''

    for i in range(0, len(images_array), max_num):
        content = [{
            "type": "text",
            "text": f"Describe what’s likely going on in frames {i} to {i + max_num}."
        }] + images_array[i:i + max_num]

        response = client.chat.completions.create(
            model="gpt-4-vision-preview",
            messages=[{"role": "user", "content": content}],
            max_tokens=1000,
        )
        frame_text += str(response.choices[0].message.content).strip('`')  
        print(f"{i} to {i+15} frames")

    return frame_text


In [ ]:
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

image_path = "./teddy.jpeg"

# Getting the base64 string
base64_image = encode_image(image_path)
response = client.chat.completions.create(
  model="gpt-4-vision-preview",
  messages=[
    {
      "role": "user",
      "content": [

        {"type": "text", "text": "what gender are they."},
        # {"type":"text", f"text": "this is the transcript of the frames - {cssx}"},
        {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/jpeg;base64,{base64_image}"
          },
        },
      ],
    }
  ],
  max_tokens=1000,
)

 

In [ ]:

response.choices[0].message.content

In [ ]:
eval(response.choices[0].message.content.strip('`').strip('json').strip('\n'))

In [ ]:
import cv2

def extract_frames(video_path, frame_interval=5):
    vidcap = cv2.VideoCapture(video_path)
    success, image = vidcap.read()
    count = 0
    frames = []

    while success:
        if count % (frame_interval * int(vidcap.get(cv2.CAP_PROP_FPS))) == 0:
            frames.append(image)
        success, image = vidcap.read()
        count += 1

    return frames

def create_grid(frames, grid_size):
    grid = None
    rows, cols = grid_size
    frame_height, frame_width, _ = frames[0].shape

    for r in range(rows):
        row = frames[r * cols:(r + 1) * cols]
        # Resize frames to ensure they have the same dimensions
        row_resized = [cv2.resize(frame, (frame_width, frame_height)) for frame in row]
        row_concatenated = cv2.hconcat(row_resized)
        if grid is None:
            grid = row_concatenated
        else:
            grid = cv2.vconcat([grid, row_concatenated])

    return grid

def save_grid(grid, output_path):
    cv2.imwrite(output_path, grid)

def generate_grids(video_path, output_folder, frame_interval=5, max_frames_per_grid=12):
    frames = extract_frames(video_path, frame_interval)
    num_frames = len(frames)
    num_grids = (num_frames - 1) // max_frames_per_grid + 1

    for i in range(num_grids):
        start_idx = i * max_frames_per_grid
        end_idx = min((i + 1) * max_frames_per_grid, num_frames)
        grid_frames = frames[start_idx:end_idx]
        grid_size = (min(len(grid_frames), 3), min(len(grid_frames), 4))
        grid = create_grid(grid_frames, grid_size)
        save_path = f"{output_folder}/grid_{i + 1}.jpg"
        save_grid(grid, save_path)

if __name__ == "__main__":
    video_path = "./video.mp4"
    output_folder = "./output/grids"
    frame_interval = 5  # 5 seconds
    max_frames_per_grid = 12

    generate_grids(video_path, output_folder, frame_interval, max_frames_per_grid)


In [41]:
text = """
okay hey so this platform is called retract especially we decide to create a new way for students to learn and a new way for students to stay on top of your studies and now I'll show you a demo so first of all with this platform you can have life transcriptions of your lectures you can have live stories of your lectures of what's going on in lecture like basically fine and lecture you know something to work on this next week you know silica to do list I know so you know cuz I said it's going to next week that can also be turned into a calendar like you can easily just like you button and I guess turned into an event on your calendar that's amazing Teddy first off I want to ask you do you know that tomorrow at April 7th at 6:00 p.m. the hackathon will be over and you have to submit your final presentation yeah yeah I know yeah I know I know okay a couple of questions that I have first off this is an amazing platform that you guys created I want to know what are some potential stakeholders that would be using this platform other than just students it's good that we found it now though what does the button change anyway back to what you were saying okay so yeah I used to stop I don't know I'll just copy this can you type in there cracked way before you wait it's just so let me add something to it so yeah this software is perfect for all types of people especially people who are in the workforce things like Consultants Bankers investment bankers and even teachers it can be applicable to anyone who needs to go into a meeting or is very lazy just so you can have a question I'm joking last night it's my house JoJo Joanna
"""

In [53]:
#txt =" So this our platform is called Retract, and Retraff is actually going to redefine the ways to this right. So what we decided to do, we have like multiple stuff. Our first of all is what we decide to do. So that's different, first of all, you know, a feature whereby students can, you know, have live transcript of live lectures. Let's say, for example, they're not enough, like with this platform, they can stay on track, like, what's you can see visibly, they also get like, they also get like the, the, the, the, the, the... like the summary of what's going on in the lecture real time they also get like an action action like key stuff as we mentioned and also doing it like a to do this on like let's say okay you have to do that and also this also serves as you know this is like a digital source that we're really used like you know creates very well structured the mission note and just like practice quizzes. in the hackathon I have a couple follow-up questions that if you would in mind I would like you to explain The first question is how did you guys get go about for the idea and what kind of codes did you use to create this? So personally I have a hard time to like constriction and class like because like it's just like there's just so much to do like this weird but"
staxx = str(
  {'summary': "The conversation introduces 'Retract', a platform designed to enhance students' learning and organizational skills. It features live transcriptions and summaries of lectures, along with a to-do list functionality that integrates with calendar events. The platform is also deemed valuable for professionals such as consultants and investment bankers. A demo was shown, emphasizing its practical applications. The discussion briefly touches on a hackathon deadline.",
 'action_items': ['Show demo of Retract platform',
  'Work on integration with calendar',
  'Prepare for hackathon submission'],
 'todo': ['Integrate to-do list with calendar'],
 'questions': {'Who are potential stakeholders for the platform?': 'Besides students, potential stakeholders include professionals like consultants, bankers, investment bankers, and even teachers. The platform is versatile, catering to anyone who attends meetings or needs an efficient way to organize their tasks and notes.'},
 'reminders': {'description': 'Hackathon submission deadline',
  'time': '6:00 PM',
  'day': '7',
  'year': '2023',
  'month': 'April'}}
)
response = client.chat.completions.create(
    model="gpt-4-turbo-preview",
messages=[
    {"role": "system", "content": "You are a helpful assistant that returns summary, actions items, todo, questions, reminders (such as date)."},
    {"role": "user", "content": f"this is the transcript of the conversation -{text}"},
    {"role": "system", "content": "Please summarize the conversation and provide action items, todo, questions, and reminders in this json format. {'summary: , action_items: , todo: , questions: , reminders: }"},
    {"role": "user", "content": "the summary should be conscise and to the point, the action items should be an array of short key stuff mentioned, the todo should be only key details mentioned in the transcript of things to do stated by the speaker do not make up todos, the questions should only key questions that what asked in the transcript, the reminders should be clear and actionable, questions should be in hashmap, have the question and the answer to the question ensure to task inivatatin and not just answer the question, but say why & how, be creative and innovative the answers, make the reminders be in a hashmap, description,time,day,year,month."},
        {"role": "user", "content": f"this prior and data of the converation leading up to present transcript - {staxx} keep the summary in arrays, of prior converstiaons and the current conversation"},

    {"role": "system", "content": "only add the the overall hashmap, do not take anything out "},

 
  ],
    response_format={ "type": "json_object" },
    max_tokens=1000,
)

In [54]:
ass = eval(response.choices[0].message.content)
ass

{'summary': ["The conversation introduces 'Retract', a platform designed to enhance students' learning and organizational skills. It features live transcriptions and summaries of lectures, along with a to-do list functionality that integrates with calendar events. The platform is also deemed valuable for professionals such as consultants and investment bankers. A demo was shown, emphasizing its practical applications. The discussion briefly touches on a hackathon deadline.",
  "In the follow-up discussion, a demonstration of 'Retract' was further elaborated, highlighting its capabilities such as live transcriptions of lectures, story summaries of lectures, and an innovative feature that converts tasks into calendar events. The conversation also sheds light on the platform's broader applicability beyond students, suggesting its relevance for various professionals. A reminder about the hackathon submission deadline is reiterated, and questions are raised regarding potential stakeholders 

In [58]:
ass['summary']

["The conversation introduces 'Retract', a platform designed to enhance students' learning and organizational skills. It features live transcriptions and summaries of lectures, along with a to-do list functionality that integrates with calendar events. The platform is also deemed valuable for professionals such as consultants and investment bankers. A demo was shown, emphasizing its practical applications. The discussion briefly touches on a hackathon deadline.",
 "In the follow-up discussion, a demonstration of 'Retract' was further elaborated, highlighting its capabilities such as live transcriptions of lectures, story summaries of lectures, and an innovative feature that converts tasks into calendar events. The conversation also sheds light on the platform's broader applicability beyond students, suggesting its relevance for various professionals. A reminder about the hackathon submission deadline is reiterated, and questions are raised regarding potential stakeholders outside the s

In [59]:
!python3 start.py

Python(11606) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


                             # STARTAPI # 

        | @author: teddyoweh                                     | 
        | @website: https://teddyoweh.net:                       | 
        | @github: https://github.com/teddyoweh                  | 
        | @twitter: https://twitter.com/tedddyoweh               | 
        | @mail: teddy@teddyoweh.net                             | 
        | @message: reach out for contributions or questions ;)! | 
        


-[#] Building None FLASK API CodeBase
Traceback (most recent call last):
  File "/Users/teddyoweh/Documents/GitHub/retrak/ml-microservice/start.py", line 2, in <module>
    startapi()
  File "/Users/teddyoweh/Library/Python/3.9/lib/python/site-packages/startapi/startapi.py", line 46, in __init__
    self._build('flaskboilerplate')
  File "/Users/teddyoweh/Library/Python/3.9/lib/python/site-packages/startapi/startapi.py", line 73, in _build
    os.chdir(self.apiname)
TypeError: chdir: path should be string, bytes, os.PathLike or int